In [1]:
# Imports
import torch
import numpy as np

In [2]:
from data_loader import load_datasets

x_train, y_train = load_datasets()
x_test,  y_test = load_datasets(train_or_test_folder="Test_Alphabet")

print(f"Train dataset count: {len(x_train)} \nTest dataset count: {len(x_test)} ")

Train dataset count: 24300 
Test dataset count: 2700 


**One-Hot Coding**

In [3]:
from data_loader import alphabet_to_num

def one_hot_coding(unique_labels, data):
    y_oh = torch.zeros(len(data),unique_labels)
    for i, class_word in enumerate(data):
        index = alphabet_to_num(class_word)
        y_oh[i][index] = 1

    return y_oh

y_train_oh = one_hot_coding(len(np.unique(y_train)),y_train)
y_test_oh = one_hot_coding(len(np.unique(y_train)),y_test)

In [4]:
x_train[0].shape,y_train.shape

((3, 50, 50), (24300,))

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=5, padding=2)
        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=4, stride=4)
        # Second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=5, out_channels=15, kernel_size=5, padding=2)
        # Another max pooling layer
        self.pool2 = nn.MaxPool2d(kernel_size=4, stride=4)
        # Fully connected layer
        self.fc1 = nn.Linear(15 * 3 * 3, 27)  # 3x3 is the size after two max pooling layers

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 15 * 3 * 3)  # Flatten the output of conv2 layer
        x = self.fc1(x)
        return x

# Instantiate the model
model = ConvNet()

# Print the model architecture
print(model)


ConvNet(
  (conv1): Conv2d(3, 5, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(5, 15, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool2): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=135, out_features=27, bias=True)
)


In [9]:
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(),lr=0.01)
batch_size = 32

train_loader = DataLoader(TensorDataset(torch.tensor(x_train),torch.tensor(y_train_oh)), 
                          batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(torch.tensor(x_test),torch.tensor(y_test_oh)), 
                          batch_size=batch_size)

/tmp/ipykernel_21836/1567307064.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_loader = DataLoader(TensorDataset(torch.tensor(x_train),torch.tensor(y_train_oh)),
/tmp/ipykernel_21836/1567307064.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_loader = DataLoader(TensorDataset(torch.tensor(x_test),torch.tensor(y_test_oh)),


In [12]:
def train_model(model,criterion,optimizer,num_epochs=50):
    for epoch in range(num_epochs):
        running_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}")


train_model(model=model, criterion=criterion, optimizer=optimizer)
        


        

Epoch [1/50], Train Loss: 2.5124
Epoch [2/50], Train Loss: 1.4197
Epoch [3/50], Train Loss: 1.2344
Epoch [4/50], Train Loss: 1.1528
Epoch [5/50], Train Loss: 1.1014
Epoch [6/50], Train Loss: 1.0685
Epoch [7/50], Train Loss: 1.0547
Epoch [8/50], Train Loss: 1.0469
Epoch [9/50], Train Loss: 1.0182
Epoch [10/50], Train Loss: 0.9953
Epoch [11/50], Train Loss: 0.9676
Epoch [12/50], Train Loss: 0.9148
Epoch [13/50], Train Loss: 0.9078
Epoch [14/50], Train Loss: 0.8927
Epoch [15/50], Train Loss: 0.8863
Epoch [16/50], Train Loss: 0.8853
Epoch [17/50], Train Loss: 0.8815
Epoch [18/50], Train Loss: 0.8656
Epoch [19/50], Train Loss: 0.8740
Epoch [20/50], Train Loss: 0.9277
Epoch [21/50], Train Loss: 0.9316
Epoch [22/50], Train Loss: 0.8772
Epoch [23/50], Train Loss: 0.8375
Epoch [24/50], Train Loss: 0.8123
Epoch [25/50], Train Loss: 0.8409
Epoch [26/50], Train Loss: 0.7971
Epoch [27/50], Train Loss: 0.7840
Epoch [28/50], Train Loss: 0.7770
Epoch [29/50], Train Loss: 0.7677
Epoch [30/50], Train Lo

In [14]:
def test_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            _, labels_indices = torch.max(labels, 1)  # Convert one-hot labels to indices
            total += labels.size(0)
            correct += (predicted == labels_indices).sum().item()
    accuracy = correct / total
    print(f"Accuracy on test set: {accuracy:.4f}")


test_model(model=model,test_loader=test_loader)

Accuracy on test set: 0.7930
